In [1]:
import pandas as pd
from datetime import timedelta


In [2]:
# Load data
users = pd.read_csv("../data/users.csv", parse_dates=["registration_date"])
sessions = pd.read_csv("../data/sessions.csv", parse_dates=["session_date"])
purchases = pd.read_csv("../data/purchases.csv", parse_dates=["purchase_date"])
attribution = pd.read_csv("../data/attribution.csv", parse_dates=["install_date"])

In [3]:
# Merge user-channel info
users = users.merge(attribution, on="user_id", how="left")

In [6]:
# Calculate ARPU per user
rev_per_user = purchases.groupby("user_id")["amount_usd"].sum().reset_index()
users = users.merge(rev_per_user, on="user_id", how="left")
users["amount_usd"] = users["amount_usd"].fillna(0)  

In [7]:
# D1 / D7 Retention hesaplama fonksiyonu
def calc_retention(df, days):
    retained = []
    for _, row in df.iterrows():
        user_id = row["user_id"]
        reg_date = row["registration_date"]
        target_date = reg_date + timedelta(days=days)
        if user_id in sessions[sessions["session_date"] == target_date]["user_id"].values:
            retained.append(1)
        else:
            retained.append(0)
    return retained

users["D1_retained"] = calc_retention(users, 1)
users["D7_retained"] = calc_retention(users, 7)


In [8]:
# Session count per user
session_counts = sessions.groupby("user_id").size().reset_index(name="session_count")
users = users.merge(session_counts, on="user_id", how="left")
users["session_count"] = users["session_count"].fillna(0)

In [9]:
# Monetizing user flag
users["is_payer"] = users["amount_usd"] > 0

In [10]:
# Kanal bazında özetleme
channel_stats = users.groupby("acquisition_channel").agg({
    "user_id": "count",
    "amount_usd": "mean",
    "D1_retained": "mean",
    "D7_retained": "mean",
    "session_count": "mean",
    "is_payer": "mean"
}).rename(columns={
    "user_id": "user_count",
    "amount_usd": "ARPU",
    "D1_retained": "D1_retention",
    "D7_retained": "D7_retention",
    "session_count": "avg_session",
    "is_payer": "conversion_rate"
})

In [11]:
channel_stats = channel_stats.round(3)
print(channel_stats.reset_index())

  acquisition_channel  user_count   ARPU  D1_retention  D7_retention  \
0        Facebook Ads        2039  2.644         0.693         0.456   
1          Google Ads        1515  2.611         0.711         0.426   
2          TikTok Ads         953  2.864         0.718         0.466   
3           Unity Ads         493  3.352         0.718         0.450   

   avg_session  conversion_rate  
0       12.757            0.242  
1       12.396            0.231  
2       12.827            0.256  
3       12.552            0.266  
